## 1. Data pre-processing

In [1]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

### 1.1 Train/Dev/Test split

In [2]:
from sklearn.model_selection import train_test_split

The dataset has been already splitted into train and test sets, with a ratio of about 70/30%. 

For our classification task, in order to fine tune the hyperparameters of the model and select the best features from the data, we further split those sets. So, we end up with train, evaluation and test sets of about 60/20/20% of the overall data, respectively.

In [3]:
dataset = pd.read_csv('./data/dataset.csv')

In [4]:
# take 2/5 of the train set for cross validation and test
train_set, dev_test_set = train_test_split(dataset, test_size=0.4, random_state=12)

# take 1/2 of the test set for cross validation
test_set, dev_set = train_test_split(dev_test_set, test_size=0.5, random_state=12)

In [5]:
train_set.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
6505,0.278909,-0.016122,-0.108804,-0.991421,-0.958884,-0.943642,-0.992854,-0.959690,-0.944281,-0.926421,...,-0.604272,-0.135149,-0.006428,-0.840651,0.548457,-0.690776,0.291574,0.110070,28,SITTING
4384,0.278544,-0.017497,-0.111402,-0.997560,-0.980832,-0.987017,-0.997578,-0.979116,-0.988149,-0.942968,...,-0.919445,-0.089110,-0.044198,-0.607149,0.601884,-0.817034,0.214795,-0.032885,22,STANDING
7480,0.243931,-0.004004,-0.122676,-0.092084,0.010789,0.199388,-0.193510,-0.041149,0.186226,0.315973,...,-0.639589,0.368217,0.310372,0.969768,-0.504041,-0.496266,0.268661,0.327231,2,WALKING_DOWNSTAIRS
5960,0.278354,-0.016345,-0.111278,-0.985230,-0.988805,-0.990760,-0.985383,-0.987864,-0.990653,-0.930099,...,-0.950759,0.332895,0.404353,0.222148,0.056738,0.604472,-0.363929,-0.648498,27,LAYING
10196,0.291115,-0.014618,-0.112456,-0.968179,-0.982618,-0.983055,-0.967095,-0.982565,-0.980663,-0.916381,...,-0.940910,-0.090369,0.008545,0.387679,-0.204587,0.449349,-0.501789,-0.490746,24,LAYING


### 1.2 Feature selection

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

The feature selection has the purpose to retain from the data only those features that are the most relevant, i.e. useful, purging the data from those which add few or no significant improvement.

Dimensionality reduction is different from feature selection, cause it projects the data in a new space, giving as a result a set of new features, whereas feature selection filters out the original features.

Feature selection is carried out by statical analyses, such as correlation analysis. For classification tasks on numerical data the ANOVA statistical model is known to be quite good. Then, we leverage its implementation in sklearn.

In [7]:
fs = SelectKBest(score_func=f_classif, k=400)

In [8]:
y_train_set = train_set[['Activity']].values.ravel()
x_train_set = fs.fit_transform(train_set.drop(['Activity','subject'], axis=1).values, y_train_set)

x_dev_set = fs.transform(dev_set.drop(['Activity','subject'], axis=1))
y_dev_set = dev_set[['Activity']].values.ravel()

x_test_set = fs.transform(test_set.drop(['Activity','subject'], axis=1))
y_test_set = test_set[['Activity']].values.ravel()

At this point, we separate the numeric values from the class labels, dropping the user ID which is useless in our analysis.

### 1.3 Dimension reduction

In [9]:
from sklearn.decomposition import TruncatedSVD

In [10]:
svd = TruncatedSVD(n_components=200)
x_train_set = svd.fit_transform(x_train_set)
x_dev_set = svd.transform(x_dev_set)

In [11]:
np.sum(svd.explained_variance_ratio_) * 100

99.93919674583786

### 1.4 Encoding Labels

Encodes the activity labels to numerical labels.

In [12]:
from sklearn import preprocessing


le = preprocessing.LabelEncoder()
y_train_set = le.fit_transform(y_train_set)

le = preprocessing.LabelEncoder()
y_dev_set = le.fit_transform(y_dev_set)

le = preprocessing.LabelEncoder()
y_test_set = le.fit_transform(y_test_set)

See the actual corresponding classes.

In [13]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'LAYING': 0, 'SITTING': 1, 'STANDING': 2, 'WALKING': 3, 'WALKING_DOWNSTAIRS': 4, 'WALKING_UPSTAIRS': 5}


# 2. Model tuning

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

In [15]:
svclassifier = SVC(kernel='rbf', C=100.0)
svclassifier.fit(x_train_set, y_train_set)

SVC(C=100.0)

In [16]:
y_pred = svclassifier.predict(x_dev_set)

In [17]:
print(confusion_matrix(y_dev_set,y_pred))
print(classification_report(y_dev_set,y_pred))

[[391   0   0   0   0   0]
 [  0 331  17   0   0   0]
 [  0  12 353   0   0   0]
 [  0   0   0 367   0   0]
 [  0   0   0   0 289   0]
 [  0   0   0   0   0 300]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       391
           1       0.97      0.95      0.96       348
           2       0.95      0.97      0.96       365
           3       1.00      1.00      1.00       367
           4       1.00      1.00      1.00       289
           5       1.00      1.00      1.00       300

    accuracy                           0.99      2060
   macro avg       0.99      0.99      0.99      2060
weighted avg       0.99      0.99      0.99      2060



In [18]:
y_pred = svclassifier.predict(x_test_set)

ValueError: X.shape[1] = 400 should be equal to 200, the number of features at training time

In [249]:
print(confusion_matrix(y_test_set,y_pred))
print(classification_report(y_test_set,y_pred))

[[399   0   0   0   0   0]
 [  0 288  72   0   0   0]
 [  0  83 298   0   0   0]
 [  0   0   0 318   1   6]
 [  0   0   0   1 262   5]
 [  0   0   0   8   1 318]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       399
           1       0.78      0.80      0.79       360
           2       0.81      0.78      0.79       381
           3       0.97      0.98      0.98       325
           4       0.99      0.98      0.98       268
           5       0.97      0.97      0.97       327

    accuracy                           0.91      2060
   macro avg       0.92      0.92      0.92      2060
weighted avg       0.91      0.91      0.91      2060



In [19]:

def select_features(train_set, dev_set, n_features):
    fs = SelectKBest(score_func=f_classif, k=n_features)
    
    x_train_set = fs.fit_transform(train_set.drop(['Activity','subject'], axis=1).values, y_train_set)

    x_dev_set = fs.transform(dev_set.drop(['Activity','subject'], axis=1))
    
    return x_train_set, x_dev_set


def reduce_features(x_train_set, x_dev_set, n_components):
    svd = TruncatedSVD(n_components=n_components)
    
    x_train_set = svd.fit_transform(x_train_set)
    
    x_dev_set = svd.transform(x_dev_set)
    
    return np.vstack((x_train_set, x_dev_set))


def tune_model(train_set, dev_set, estimator, params):
    log_result = {}
    test_fold = np.append(np.full((train_set.shape[0],), -1, dtype=int), np.full((dev_set.shape[0],), 0, dtype=int))
    ps = PredefinedSplit(test_fold)
    y_train_dev = np.vstack((train_set[['Activity']].values, dev_set[['Activity']].values)).ravel()
    for n_features in tqdm((50, 100, 300, 400, 561)):
        log_result[n_features] = {}
        x_train_set, x_dev_set = select_features(train_set, dev_set, n_features)
        for n_components in range(10, n_features, 50):
            train_dev_set = reduce_features(x_train_set, x_dev_set, n_components)
            grid = GridSearchCV(estimator=estimator, param_grid=params, scoring='accuracy', cv=ps)
            grid.fit(train_dev_set, y_train_dev)
            log_result[n_features][n_components] = (grid.best_score_, grid.best_params_)
    return log_result
            

In [20]:
params={'kernel':['linear','rbf'],'C':[1,10,100],'gamma':[1e-2,1e-3,1e-4]}
result = tune_model(train_set, dev_set, SVC(), params)

100%|██████████| 5/5 [25:36<00:00, 307.27s/it]


In [29]:
max_acc = []
index = []
for n_features in (50, 100, 300, 400, 561):
    for n_components in range(10, n_features, 50):
        max_acc.append(result[n_features][n_components][0])
        index.append((n_features, n_components))
            
i = np.argsort(np.array(max_acc))
#print(np.array(max_acc)[i[-25:]])
#print(np.array(index)[i[-25:]])
print(index[i[-5]], result[561][360])

(561, 360) (0.9912621359223301, {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'})


## GDA Multi-Classification OVA

Compute the phi y parameter, now we are inserting the methods that estimate the parameters for the GDA analysis.

In [16]:
def compute_phi(y):
    return np.mean(y)

Compute the mean for the first vector and second vector

In [17]:
def compute_mu0(x, y):
    indicator_f = 1-y.reshape(y.shape[0], 1)

    sum_x = 0
    for i in range(0, x.shape[0]):
        sum_x +=indicator_f[i] * x[i]
    
    return sum_x / np.sum(indicator_f)


def compute_mu1(x, y):
    indicator_f = y.reshape(y.shape[0], 1)
    
    sum_x = 0
    for i in range(0, x.shape[0]):
        sum_x += indicator_f[i] * x[i]
    
    return sum_x / np.sum(indicator_f)

Compute sigma

In [19]:
def compute_sigma(x, y, mu0, mu1):
    my_sum = 0
    for i in range(0, x.shape[0]):
        if y[i]==0:
            my_sum+=(x[i] - mu0).reshape(x.shape[1], 1) @ (x[i] - mu0).reshape(x.shape[1], 1).T
        else:
            my_sum+=(x[i] - mu1).reshape(x.shape[1], 1) @ (x[i] - mu1).reshape(x.shape[1], 1).T
            
    return (1 / x.shape[0]) * (my_sum)

Write the probability of x give y.

In [20]:
# computing p(x|y) for the Bayes rule
def p_x_given_y(x,mu,sigma):  
    d = x.shape[0]
    return (1 / (((2*np.pi)**(d/2)) * (np.linalg.det(sigma)**0.5))) * np.exp(-0.5*(x - mu) @ np.linalg.inv(sigma) @ (x - mu))

# Function p(y) for applying the Bayes rule
def p_y(y,phi):
    if abs(y)==1: return phi
    else: return 1 - phi

In [21]:
# Now estimate the GDA parameters and start for one vs all
selected_example=1
print('Selected example =', x_test_set[selected_example,:], "\n")

for label in [0, 1, 2, 3, 4, 5]:
    y = y_train_set.copy()
    y = np.where(y != label, -1, y) # the rest is set to -1
    y = np.where(y == label, 0, y) # the value point that we want to compare with the rest is set to zero
    y = abs(y)
    
    phi = compute_phi(y)
    mu0 = compute_mu0(x_train_set, y)
    mu1 = compute_mu1(x_train_set, y)
    sigma = compute_sigma(x_train_set, y, mu0, mu1)

    # compute p(y=l|x) ~ p(x|y=0)*p(y=0)  &  p(y=1|x) ~ p(x|y=1)*p(y=1) where l is one class

    # y=0
    print('p(y=' + str(label) + '|x) ~', p_x_given_y(x_test_set[selected_example,:],mu0,sigma)*p_y(0, phi))

    # y=1
    print('p(y=1|x) ~', p_x_given_y(x_test_set[selected_example,:],mu1,sigma)*p_y(1, phi))

Selected example = [0.928801   0.85808986 0.94499787 0.80917682 0.65810016 0.72029408
 0.55315077 0.53497811 0.62244386 0.38211905] 

p(y=0|x) ~ 1.9866046722794367e-28
p(y=1|x) ~ 893783.7291330532
p(y=1|x) ~ 1814.5070550259309
p(y=1|x) ~ 231886.4838062258
p(y=2|x) ~ 1626.5160686718027
p(y=1|x) ~ 247106.33405376732
p(y=3|x) ~ 134223.67329465155
p(y=1|x) ~ 53830.9856840042
p(y=4|x) ~ 5350.579906360915
p(y=1|x) ~ 183700.30797465434
p(y=5|x) ~ 40582.90941693707
p(y=1|x) ~ 137513.41795511777
